<a href="https://colab.research.google.com/github/KYluiza/Imobili-ria-figueira-BRZ/blob/main/WebScrapping_Imobiliaria_figueira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install requests
!pip install beautifulsoup4
!pip install selenium

!pip install webdriver_manager
!pip install --upgrade webdriver-manager
!pip install dash plotly pandas

In [10]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [11]:
URL_BASE = "https://www.imobiliariafigueira.com.br/imoveis/para-alugar/apartamento/barretos"


In [12]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
}
# Faça a solicitação HTTP e obtenha o HTML da página
response = requests.get(URL_BASE, headers=headers)

In [13]:
# Verifique se a solicitação foi bem-sucedida (código 200)
if response.status_code == 200:
    # Use BeautifulSoup para analisar o conteúdo HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontre todos os elementos com a classe 'card-with-buttons__code'
    codes = soup.find_all('p', class_='card-with-buttons__code')

    # Crie uma lista para armazenar as informações
    property_list = []

    # Para cada código, encontre as informações relevantes e adicione à lista
    for code in codes:
        # Encontre o tipo do imóvel
        type_element = code.find_next('p', class_='card-with-buttons__title')

        # Encontre a localização
        location_element = code.find_next('h2', class_='card-with-buttons__heading')

        # Encontre o valor do condomínio (se existir)
        condo_element = code.find_next('p', class_='card-with-buttons__condo')

        # Encontre a lista de informações adicionais
        additional_info_element = code.find_next('ul')

        # Se a lista de informações adicionais existir, colete os dados
        if additional_info_element:
            additional_info_items = additional_info_element.find_all('li')
            additional_info = [item.text.strip() for item in additional_info_items]
        else:
            additional_info = ['N/A']

        # Encontre a seção com informações de venda e locação
        baseboard_section = code.find_next('div', class_='card-with-buttons__baseboard')

        # Inicialize os valores como N/A
        venda_value = 'N/A'
        locacao_value = 'N/A'

        # Se a seção de baseboard existir
        if baseboard_section:
            # Encontre todos os contêineres de valor dentro da seção de baseboard
            value_containers = baseboard_section.find_all('div', class_='card-with-buttons__value-container')

            # Para cada contêiner de valor
            for value_container in value_containers:
                # Encontre o título e o valor
                title = value_container.find('p', class_='card-with-buttons__value-title').text.strip()
                value = value_container.find('p', class_='card-with-buttons__value').text.strip()

                # Atribua aos campos apropriados com base no título
                if title == 'Locação':
                    locacao_value = value
                elif title == 'Venda':
                    venda_value = value

        # Use .text se o elemento existir, senão, use 'N/A'
        type_text = type_element.text.strip() if type_element else 'N/A'
        location_text = location_element.text.strip() if location_element else 'N/A'
        condo_text = condo_element.text.strip() if condo_element else 'N/A'

        # Adicione as informações à lista
        property_data = {
            "Código": code.text.strip(),
            "Tipo do Imóvel": type_text,
            "Localização": location_text,
            "Condomínio": condo_text,
            "Venda": venda_value,
            "Locação": locacao_value,
            "Informações Adicionais": additional_info        }

        property_list.append(property_data)

    # Imprima a lista de propriedades
    for property_data in property_list:
        print(property_data)

else:
    print(f'Falha na solicitação. Código de status: {response.status_code}')


{'Código': 'AP1304-FIA', 'Tipo do Imóvel': 'Apartamento', 'Localização': 'Cristiano de Carvalho - Barretos - SPPortal Quinta Das Pitangueiras', 'Condomínio': 'Portal Quinta Das Pitangueiras', 'Venda': 'R$ 350.000', 'Locação': 'R$ 1.900/mês', 'Informações Adicionais': ['55 m²', '2 Quartos', '1 Suíte', '2 Banheiros']}
{'Código': 'AP1287-FIA', 'Tipo do Imóvel': 'Apartamento', 'Localização': 'Cristiano de Carvalho - Barretos - SPPortal Quinta Flor de Cerejeira', 'Condomínio': 'Portal Quinta Flor de Cerejeira', 'Venda': 'N/A', 'Locação': 'R$ 1.500/mês', 'Informações Adicionais': ['48 m²', '2 Quartos', '1 Banheiro', '1 Vaga']}
{'Código': 'AP1305-FIA', 'Tipo do Imóvel': 'Apartamento', 'Localização': 'Cristiano de Carvalho - Barretos - SPPortal Quinta Das Pitangueiras', 'Condomínio': 'Portal Quinta Das Pitangueiras', 'Venda': 'R$ 248.000', 'Locação': 'R$ 1.500/mês', 'Informações Adicionais': ['52 m²', '2 Quartos', '1 Banheiro']}
{'Código': 'AP1299-FIA', 'Tipo do Imóvel': 'Apartamento', 'Locali

In [14]:
tabela_imobiliaria = pd.DataFrame(property_list)

tabela_imobiliaria


,Código,Tipo do Imóvel,Localização,Condomínio,Venda,Locação,Informações Adicionais
0,AP1304-FIA,Apartamento,Cristiano de Carvalho - Barretos - SPPortal Qu...,Portal Quinta Das Pitangueiras,R$ 350.000,R$ 1.900/mês,"[55 m², 2 Quartos, 1 Suíte, 2 Banheiros]"
1,AP1287-FIA,Apartamento,Cristiano de Carvalho - Barretos - SPPortal Qu...,Portal Quinta Flor de Cerejeira,N/A,R$ 1.500/mês,"[48 m², 2 Quartos, 1 Banheiro, 1 Vaga]"
2,AP1305-FIA,Apartamento,Cristiano de Carvalho - Barretos - SPPortal Qu...,Portal Quinta Das Pitangueiras,R$ 248.000,R$ 1.500/mês,"[52 m², 2 Quartos, 1 Banheiro]"
3,AP1299-FIA,Apartamento,Doutor Paulo Prata - Barretos - SP,Edifício Residencial Hortênsias,N/A,R$ 2.000/mês,"[60 m², 2 Quartos, 1 Suíte, 2 Banheiros]"
4,AP1307-FIA,Apartamento,Aeroporto - Barretos - SPEdifício Residencial ...,Edifício Residencial Hortênsias,N/A,R$ 2.200/mês,"[44.5 m², 1 Quarto, 1 Banheiro, 1 Vaga]"
5,KN0107-FIA,Kitnet,Jardim Universitário - Barretos - SP,N/A,N/A,R$ 950/mês,"[50 m², 1 Quarto, 1 Banheiro, 1 Vaga]"
6,AP0588-FIA,Apartamento,Fortaleza - Barretos - SP,N/A,N/A,R$ 1.150/mês,"[40 m², 2 Quartos, 1 Banheiro, 1 Vaga]"
7,AP1306-FIA,Apartamento,Jardim Soares - Barretos - SP,N/A,N/A,R$ 1.200/mês,"[48 m², 2 Quartos, 1 Banheiro, 1 Vaga]"
8,AP0592-FIA,Apartamento,Fortaleza - Barretos - SP,N/A,N/A,R$ 960/mês,"[55 m², 1 Quarto, 1 Banheiro, 1 Vaga]"
9,AP1279-FIA,Apartamento,Jardim Universitário - Barretos - SP,N/A,N/A,R$ 1.700/mês,"[58 m², 2 Quartos, 1 Suíte, 1 Banheiro, 1 Vaga]"
